In [1]:
#부동산 소유권 취득현황

In [6]:
import requests
import json
import datetime as dt
from hdfs import InsecureClient
from json import dumps
import pandas as pd

In [7]:
client = InsecureClient('http://localhost:9870', user='big')

In [18]:
JDBC = {
    'url':'jdbc:oracle:thin:@realestate_high?TNS_ADMIN=/home/big/study/db/Wallet_REALESTATE'
    ,'props':{
        'user':'dw_realestate',
        'password':'123qwe!@#QWE'
    }   
}

## 2017-2022 데이터 수집

In [24]:
#월 리스트 생성
deal_y = ['201'+str(i) for i in range(7,10)]
deal_y.append('2020')
deal_y.append('2021')
deal_y.append('2022')

deal_ymd = []
for year in deal_y:
        for m in range(1,13):
            if m < 10:
                m = str(0) + str(m)
            if year+str(m) > '202209' :
                break
            deal_ymd.append(year+str(m))

deal_ymd=deal_ymd[0:1]
print(deal_ymd)

['201701']


### api실행함수/기준일자 함수정의

In [9]:
# url = "https://data.iros.go.kr/rp/ds/selectDatasetInqDetl.do"
# service_key = '746cac395d3d43f2bc4ba51f1fe56a68
file_dir = '/project/'
url = "https://data.iros.go.kr/rp/ds/selectDatasetInqDetl.do"
service_key = '746cac395d3d43f2bc4ba51f1fe56a68'

data={
    'pageIndex':'1'
    ,'sort_cond':''
    ,'sort_ord':''
    ,'ris_menu_seq':'0000000007'
    ,'title':'%EB%B6%80%EB%8F%99%EC%82%B0 %EC%86%8C%EC%9C%A0%EA%B6%8C %EC%B7%A8%EB%93%9D%ED%98%84%ED%99%A9(%EC%A7%80%EC%97%AD%2C %EC%86%8C%EC%9C%A0%EC%9E%90%EB%B3%84)'
    ,'search_date':'201701'
    ,'rdata_seq':'0000000252'
    ,'search_type':'037001'
    ,'search_word':'%EB%B6%80%EB%8F%99%EC%82%B0 %EC%86%8C%EC%9C%A0%EA%B6%8C %EC%B7%A8%EB%93%9D%ED%98%84%ED%99%A9'
    ,'pagePerCount':'10'
    ,'search_order':'031003'
    ,'dts_prmm_ofr_yn':'Y'
    ,'returnUrl':''
    ,'search_regn_name':''
    ,'search_year':''
    ,'search_mon':''

}

headers={
    'Content-Type':'application/x-www-form-urlencoded'
    ,'-User-Agent':'PostmanRuntime/7.29.2'
    ,'Accept':'*/*'
    ,'Accept-Encoding':'gzip, deflate, br'
    ,'Connection':'keep-alive'
}
    


In [25]:

for i in range(len(deal_ymd)):
    res = requests.post(url,data,headers,verify=False,timeout=10)
    res.encoding='utf-8'
    file_name = 'realestate_own' + deal_ymd[i] + '.json'
    client.write(file_dir + file_name, json.dumps(res.json(),ensure_ascii=False), overwrite=True, encoding='utf-8')

/home/big/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.iros.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## 매월 해당 데이터 추출 

In [33]:
def cal_std_month(before_month):
    x = dt.datetime.now() - dt.timedelta(30*before_month) #현재날짜-지정날짜
    year = x.year #연도
    month = x.month if x.month >= 10 else '0'+ str(x.month) #월
    return str(year) + str(month)

In [34]:
file_dir = '/project/'
url = "https://data.iros.go.kr/rp/ds/selectDatasetInqDetl.do"
service_key = '746cac395d3d43f2bc4ba51f1fe56a68'

data={
    'pageIndex':'1'
    ,'sort_cond':''
    ,'sort_ord':''
    ,'ris_menu_seq':'0000000007'
    ,'title':'%EB%B6%80%EB%8F%99%EC%82%B0 %EC%86%8C%EC%9C%A0%EA%B6%8C %EC%B7%A8%EB%93%9D%ED%98%84%ED%99%A9(%EC%A7%80%EC%97%AD%2C %EC%86%8C%EC%9C%A0%EC%9E%90%EB%B3%84)'
    ,'search_date':cal_std_month(0)
    ,'rdata_seq':'0000000252'
    ,'search_type':'037001'
    ,'search_word':'%EB%B6%80%EB%8F%99%EC%82%B0 %EC%86%8C%EC%9C%A0%EA%B6%8C %EC%B7%A8%EB%93%9D%ED%98%84%ED%99%A9'
    ,'pagePerCount':'10'
    ,'search_order':'031003'
    ,'dts_prmm_ofr_yn':'Y'
    ,'returnUrl':''
    ,'search_regn_name':''
    ,'search_year':''
    ,'search_mon':''

}

headers={
    'Content-Type':'application/x-www-form-urlencoded'
    ,'-User-Agent':'PostmanRuntime/7.29.2'
    ,'Accept':'*/*'
    ,'Accept-Encoding':'gzip, deflate, br'
    ,'Connection':'keep-alive'
}
    

In [22]:


res = requests.post(url,data,headers,verify=False,timeout=15)
file_name = 'realestate' + data['search_date'] + '.json'
client.write(file_dir + file_name, json.dumps(res.json(),ensure_ascii=False), overwrite=True, encoding='utf-8')


/home/big/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.iros.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [36]:
def realesstate_to_hdfs(client, file_dir, url, data, headers):
    res = requests.post(url,data,headers,verify=False,timeout=10)
    file_name = 'realestate_regi' + data['search_date'] + '.json'
    client.write(file_dir + file_name, json.dumps(res.json()),overwrite=True, encoding='utf-8')

realesstate_to_hdfs(client, file_dir, url, data, headers)

/home/big/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.iros.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
